In [ ]:
# import numpy as np
# import networkx as nx
# import random
# from gensim.models import Word2Vec


# class Graph():
#     def __init__(self, nx_G, is_directed, p, q):
#         self.G = nx_G
#         self.is_directed = is_directed
#         self.p = p
#         self.q = q

#     def node2vec_walk(self, walk_length, start_node):
#         '''
#         Simulate a random walk starting from start node.
#         '''
#         G = self.G
#         alias_nodes = self.alias_nodes
#         alias_edges = self.alias_edges

#         walk = [start_node]

#         while len(walk) < walk_length:
#             cur = walk[-1]
#             cur_nbrs = sorted(G.neighbors(cur))
#             if len(cur_nbrs) > 0:
#                 if len(walk) == 1:
#                     walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
#                 else:
#                     prev = walk[-2]
#                     next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
#                         alias_edges[(prev, cur)][1])]
#                     walk.append(next)
#             else:
#                 break

#         return walk

#     def simulate_walks(self, num_walks, walk_length):
#         '''
#         Repeatedly simulate random walks from each node.
#         '''
#         G = self.G
#         walks = []
#         nodes = list(G.nodes())
#         print ('Walk iteration:')
#         for walk_iter in range(num_walks):
#             print (str(walk_iter+1), '/', str(num_walks))
#             random.shuffle(nodes)
#             for node in nodes:
#                 walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

#         return walks

#     def get_alias_edge(self, src, dst):
#         '''
#         Get the alias edge setup lists for a given edge.
#         '''
#         G = self.G
#         p = self.p
#         q = self.q

#         unnormalized_probs = []
#         for dst_nbr in sorted(G.neighbors(dst)):
#             if dst_nbr == src:
#                 unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
#             elif G.has_edge(dst_nbr, src):
#                 unnormalized_probs.append(G[dst][dst_nbr]['weight'])
#             else:
#                 unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
#         norm_const = sum(unnormalized_probs)
#         normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

#         return alias_setup(normalized_probs)

#     def preprocess_transition_probs(self):
#         '''
#         Preprocessing of transition probabilities for guiding the random walks.
#         '''
#         G = self.G
#         is_directed = self.is_directed

#         alias_nodes = {}
#         for node in G.nodes():
#             unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
#             norm_const = sum(unnormalized_probs)
#             normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
#             alias_nodes[node] = alias_setup(normalized_probs)

#         alias_edges = {}
#         triads = {}

#         if is_directed:
#             for edge in G.edges():
#                 alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
#         else:
#             for edge in G.edges():
#                 alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
#                 alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

#         self.alias_nodes = alias_nodes
#         self.alias_edges = alias_edges

#         return


# def alias_setup(probs):
#     '''
#     Compute utility lists for non-uniform sampling from discrete distributions.
#     Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
#     for details
#     '''
#     K = len(probs)
#     q = np.zeros(K)
#     J = np.zeros(K, dtype=np.int)

#     smaller = []
#     larger = []
#     for kk, prob in enumerate(probs):
#         q[kk] = K*prob
#         if q[kk] < 1.0:
#             smaller.append(kk)
#         else:
#             larger.append(kk)

#     while len(smaller) > 0 and len(larger) > 0:
#         small = smaller.pop()
#         large = larger.pop()

#         J[small] = large
#         q[large] = q[large] + q[small] - 1.0
#         if q[large] < 1.0:
#             smaller.append(large)
#         else:
#             larger.append(large)

#     return J, q

# def alias_draw(J, q):
#     '''
#     Draw sample from a non-uniform discrete distribution using alias sampling.
#     '''
#     K = len(J)

#     kk = int(np.floor(np.random.rand()*K))
#     if np.random.rand() < q[kk]:
#         return kk
#     else:
#         return J[kk]

In [ ]:
import node2vec_lib
import networkx as nx

In [ ]:
def read_graph(weighted,directed,args):
    '''
    Reads the input network in networkx.
    '''
    if weighted:
        G = nx.read_edgelist(args, nodetype=int, data=(('weight',float),), create_using=nx.DiGraph())
    else:
        G = nx.read_edgelist(args.input, nodetype=int, create_using=nx.DiGraph())
        for edge in G.edges():
            G[edge[0]][edge[1]]['weight'] = 1

    if not directed:
        G = G.to_undirected()

    return G

In [ ]:
G = nx.read_edgelist('/Users/s0p00zp/Documents/main/work/edgelist.txt', nodetype=int)

In [ ]:
nx.draw(G)

In [ ]:
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

In [ ]:
G= G.to_undirected()

In [ ]:
import node2Vec1

In [ ]:
list(G.nodes())[0]

In [ ]:
[G[1][nbr]['weight'] for nbr in sorted(G.neighbors(1))]

In [ ]:
G = node2Vec1.Graph(G, False, 1., 1.)

In [ ]:
alias_nodes = {}
for node in G.nodes():
    unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
    norm_const = sum(unnormalized_probs)
    normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
    alias_nodes[node] = alias_setup(normalized_probs)

In [ ]:
probs=[0.25,0.25,0.25,0.25]
K = len(probs)
q = np.zeros(K)
J = np.zeros(K, dtype=np.int)

In [ ]:
smaller = []
larger = []
for kk, prob in enumerate(probs):
    q[kk] = K*prob
    if q[kk] < 1.0:
        smaller.append(kk)
    else:
        larger.append(kk)

In [ ]:
while len(smaller) > 0 and len(larger) > 0:
    small = smaller.pop()
    large = larger.pop()

    J[small] = large
    q[large] = q[large] + q[small] - 1.0
    if q[large] < 1.0:
        smaller.append(large)
    else:
        larger.append(large)

In [ ]:
q

In [ ]:
alias_edges = {}
triads = {}
for edge in G.edges():
    alias_edges[edge] = get_alias_edge(edge[0], edge[1])
    alias_edges[(edge[1], edge[0])] = get_alias_edge(edge[1], edge[0])

In [ ]:
alias_nodes

In [ ]:
G.preprocess_transition_probs()

In [ ]:
walks = G.simulate_walks(10, 80)

In [ ]:
def learn_embeddings(walks,dimensions,window_size,workers,iter):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    walks = [list(map(str, walk)) for walk in walks]
    model = Word2Vec(walks, size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, iter=iter)
    model.save_word2vec_format(args.output)
    
    return

In [ ]:
walks = [list(map(str, walk)) for walk in walks]
model = Word2Vec(walks, size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, iter=iter)

In [ ]:
dimensions=128
window_size=10
workers=8
iter=1
from gensim.models import Word2Vec
learn_embeddings(walks,dimensions,window_size,workers,iter)

In [ ]:
model.wv.vocab

In [ ]:
len(walks[100])

In [ ]:
settings = {
    'window_size': 2, # context window +- center word
    'n': 10, # dimensions of word embeddings, also refer to size of hidden layer
    'epochs': 50, # number of training epochs
    'learning_rate': 0.01 # learning rate
}

In [ ]:
def generate_training_data(settings, corpus):
    # Find unique word counts using dictonary
    window=settings['window_size']
    word_counts = dict()
    for row in corpus:
        for word in row:
            word_counts[word] += 1
    ## How many unique words in vocab? 9
    v_count = len(word_counts.keys())
    # Generate Lookup Dictionaries (vocab)
    words_list = list(word_counts.keys())
    # Generate word:index
    word_index = dict((word, i) for i, word in enumerate(words_list))
    # Generate index:word
    index_word = dict((i, word) for i, word in enumerate(words_list))

    training_data = []
    # Cycle through each sentence in corpus
    for sentence in corpus:
        sent_len = len(sentence)
        # Cycle through each word in sentence
        for i, word in enumerate(sentence):
            # Convert target word to one-hot
            w_target = word2onehot(sentence[i])
            # Cycle through context window
            w_context = []
            # Note: window_size 2 will have range of 5 values
            for j in range(i - window, i + window+1):
                # Criteria for context word 
                # 1. Target word cannot be context word (j != i)
                # 2. Index must be greater or equal than 0 (j >= 0) - if not list index out of range
                # 3. Index must be less or equal than length of sentence (j <= sent_len-1) - if not list index out of range 
                if j != i and j <= sent_len-1 and j >= 0:
                    # Append the one-hot representation of word to w_context
                    w_context.append(word2onehot(sentence[j]))
                    # print(sentence[i], sentence[j]) 
                    # training_data contains a one-hot representation of the target word and context words
            training_data.append([w_target, w_context])
    return np.array(training_data)


In [ ]:
alias_setup([0.25,0.25,0.25,0.25])

In [ ]:
def word2onehot(word):
    
    # word_vec - initialise a blank vector
    word_vec = [0 for i in range(0, v_count)] # Alternative - np.zeros(self.v_count)
    
    # Get ID of word from word_index
    word_index = word_index[word]
    
    # Change value from 0 to 1 according to ID of the word
    word_vec[word_index] = 1
    
    return word_vec

In [ ]:
def train(training_data):
    # Initialising weight matrices
    # getW1 - shape (9x10) and getW2 - shape (10x9)
    #self.w1 = np.array(getW1)
    #self.w2 = np.array(getW2)
    w1 = np.random.uniform(-1, 1, (self.v_count, self.n)) # weight to be supplied by souradip
    w2 = np.random.uniform(-1, 1, (self.n, self.v_count)) # weight to be supplied by souradip

In [ ]:
import node2vec
print(random.__file__)

In [ ]:
print(random.__file__)